In [1]:
#!/usr/bin/env python3
#import your libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_transaction = pd.read_csv('/Users/krahman/work/fraud_detection/data/train_transaction.csv')
train_identity = pd.read_csv('/Users/krahman/work/fraud_detection/data/train_identity.csv')
# merging dataframes 
df_train = train_transaction.merge(train_identity, on='TransactionID', how='left')

In [53]:
train_transaction.shape[0]/2

295270.0

In [3]:
train_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train_identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [5]:
# train_transaction.info();

In [6]:
# train_identity.info();

In [7]:
train_transaction_sum = train_transaction.duplicated().sum()
train_identity_sum = train_identity.duplicated().sum()
columns = train_transaction.columns
print('Train transaction duplicates: {}\nTrain identity duplicates: {} \n'.format(train_transaction_sum, train_identity_sum))
# print('Train feature columns:\n', list(columns))

Train transaction duplicates: 0
Train identity duplicates: 0 



In [8]:
# print(train_transaction.shape)
# print(train_identity.shape)

In [9]:
# df_train.shape

In [10]:
# # print(df_train.shape)
# df_fraud = df_train[df_train['isFraud']==1]
# df_no_fraud = df_train[df_train['isFraud']==0]
# print(df_fraud.shape)
# print(df_no_fraud.shape)
# # do we have more information for folks who end up having fraud issues?

In [11]:
# list_fraud = np.sum(df_fraud.isnull())
# list_nofraud = np.sum(df_no_fraud.isnull())

In [12]:
# print(sum(np.sum(df_fraud.isnull())))
# print(sum(np.sum(df_no_fraud.isnull())))

In [13]:
# # What are total null values of each...?
# df_fraud.shape[0]/sum(np.sum(df_fraud.isnull()))

In [14]:
# df_no_fraud.shape[0]/sum(np.sum(df_no_fraud.isnull()))

In [15]:
# determine what features we can drop due to too many missing features
# list_notnull = df_fraud[(df_fraud.isnull())].index
# df_fraud.loc[list_notnull,:]

In [16]:
# df_null = pd.DataFrame(np.sum(df_train.isnull()), columns=['total']).reset_index()

### Assigning columns to specific lists (cat, num, date, etc.)

In [17]:
# dropping columns with more than 50% missing data
length_df = df_train.shape[0]/2
list_temp = []
for val in df_train.columns:
    if np.sum(df_train[val].isnull()) > length_df:
        list_temp.append(val)   
df_train = df_train.drop(list_temp, axis=1)

In [18]:
# c is num, ex, how many addresses associated with card
col_c = [c for c in df_train.columns if c.startswith('C') and (len(c)==2 or len(c)==3)]
# d is num, time/days between transactions
col_d = [d for d in df_train.columns if d.startswith('D') and (len(d)==2 or len(d)==3)]
# m is date of transaction
col_m = [m for m in df_train.columns if m.startswith('M') and (len(m)==2 or len(m)==3)]
# v is num, features created by vesta such as ranking, counting. entity relationships, etc. 
col_v = [v for v in df_train.columns if v.startswith('V') and (len(v)==2 or len(v)==3 or len(v)==4)]
# i is identity information like network and digital signature associated with transaction
col_i = [i for i in df_train.columns if i.startswith('id_') and len(i)==5]
# ca is cat, card information such as card type, etc. 
col_card = [ca for ca in df_train.columns if ca.startswith('card')]

In [20]:
# D = time elapsed between each transaction, card = card information, C = counting, ie how many addresses 
# associated with card, M=True/False, V created features on ranking, counting, etc. 

# column id and target
col_id = ['TransactionID']
col_target = ['isFraud']

# converting categorical columns with numerical values to string types.
col_cat_to_obj = ['addr1','addr2','card1','card2', 'card3', 'card5']
for val in col_cat_to_obj:
    df_train[val] = df_train[val].astype(str)

# categorical columns
col_cat = ['addr1','addr2','ProductCD',"P_emaildomain"] + col_card

# C counter, D is time elapsed between transactions, V feature engineered variables by firm
col_num = ['TransactionAmt'] + col_c + col_d + col_v

# figure out how to handle this. What do these dates mean? Do certain dates have more fraud occurences?
col_date = ['TransactionDT'] 

# boolean columns. convert via dummy variable. We dont know if true/false is better than one or the other. 
col_bool = col_m

# confirming all columns are accounted for
print('Total columns: ' + str(len(col_cat + col_num + col_date + col_bool + col_id + col_i + col_target)))

# col_all = col_cat + col_num + col_date + col_bool + col_id + col_target
# columns removed dist1, dist2, R_emaildomain, DeviceInfo, DeviceType

Total columns: 220


### Filling In Missing Values

In [31]:
# 295270 50%

In [100]:
len(df_train['P_emaildomain'].unique())

60

In [102]:
class Preprocessing:
    def __init__(self):
        self.list_col = []
        self.list_total = []
        self.list_datatype = []
        self.list_unique_val = []
        self.list_unique_total = []
        
    def missing_values(self):
        for col in df_train.columns:
            total_null = np.sum(df_train[col].isnull())
            if total_null > 0:
                self.list_col.append(col)
                self.list_total.append(total_null)
                self.list_datatype.append(df_train[col].dtype)
                self.list_unique_total.append(len(df_train[col].unique()))
#                 self.list_unique_val.append(df_train[col].unique())
        print(self.list_unique_val)
        list_null = self._create_dataframe(self.list_col, 'col_name')
        list_val = self._create_dataframe(self.list_total, 'total_null')
        list_datatype = self._create_dataframe(self.list_datatype, 'datatype')
        list_unique_total = self._create_dataframe(self.list_unique_total, 'total_unique')
#         series_unique_val = self._create_dataframe(self.list_unique_val, 'unique')
#         print(series_unique_val.shape)
        df_null = self._concat_series(list_null, list_val, list_datatype, list_unique_total)
        df_null.to_csv('/Users/krahman/work/fraud_detection/saved_files/df_null.csv')
        return df_null

    def _create_dataframe(self, list_val, title):
        return pd.DataFrame(list_val, columns=[title])
    
    def _concat_series(self, null, val, datatype, total):
        return pd.concat([null, val, datatype, total],axis=1).sort_values('total_null',ascending=False)
    
    def _drop_columns(self):
        return
pp = Preprocessing()
pp.missing_values()

# NEXT, determine what to do with columns that have too many unique values... obviously.. types of solutions
# would be to put "MISSING" for those that dont have an email address... but you will need to evaluate 
# and make instead a counter of unique values, then append that and look at the CSV via google sheets. use 
# something like the code below 
# so we left off on calculating total_unique... should we create some kind of if statement that skips that col
# if there are too many unique variables? Too many unique variables indicate a col that is not categorical. 

# Planning - our preprocessing method must automatically drop missing values, but we can't do that because
# we need to see about filling them in first, then decide if we need to drop them. Right now, we need to
# create a dataframe that shows unique values for each column with missing values. 

[]


,col_name,total_null,datatype,total_unique
13,M4,281444,object,4
4,D2,280797,float64,642
15,V1,279287,float64,3
24,V10,279287,float64,6
23,V9,279287,float64,10
...,...,...,...,...
168,V295,12,float64,929
170,V297,12,float64,14
171,V298,12,float64,95
172,V299,12,float64,51


In [54]:
df_train[col_v]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,117.0,0.0,0.000000,0.000000,0.000000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,135.0,0.000000,0.000000,0.000000,50.0,1404.0,790.0,0.000000,0.000000,0.000000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,47.950001,47.950001,47.950001,0.0,0.0,0.0,0.000000,0.000000,0.000000
590536,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
590537,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
590538,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0,...,117.0,317.500000,669.500000,317.500000,0.0,2234.0,0.0,0.000000,0.000000,0.000000


In [ ]:
from housing data, create OOP method
all_cols = features.columns
cols_nan = []
cols_missing_total = []
data_type = []
unique_val = []
for col in all_cols:
    if features[col].isnull().sum() > 0:
        cols_nan.append(col)
        cols_missing_total.append(features[col].isnull().sum())
        data_type.append(features[col].dtype)
        unique_val.append(features[col].unique())
cols_nan = pd.Series(data=cols_nan)
cols_missing_total = pd.Series(data=cols_missing_total)
data_type = pd.Series(data=data_type)
unique_val = pd.Series(data=unique_val)
df_missing_values = pd.concat([cols_nan,cols_missing_total,data_type, unique_val], 
                               keys=['cols_nan','cols_missing_total','data_type','unique_val'],
                               axis=1)
df_missing_values.sort_values('cols_missing_total',ascending=False)

In [51]:
df_null_total.sort_values('total_null', ascending=False)
# np.sum(df_train['card4'].isnull())

,col_name,total_null
13,M4,281444
4,D2,280797
15,V1,279287
24,V10,279287
23,V9,279287
...,...,...
168,V295,12
170,V297,12
171,V298,12
172,V299,12


In [22]:
# sns.distplot(df_train['V1'])

In [23]:
# print(pd.DataFrame(df_train[col_num].dtypes).reset_index())
# for val in col_num:
#     val_dtype = df_train[val].dtypes
# #     print(val)
# #     print(df_train[val][0])
#     print(str(val_dtype))
# # NEXT, ensure col num and actual values look correct. 
# df_train[col_cat].dtypes
# pd.DataFrame(np.sum(train_df.isnull())).reset_index().to_csv('fraud_detection_null.csv')

In [24]:
# # dropping columns with more than 50% missing data automatically.
# length_df = df_train.shape[0]/2
# list_temp = []
# for val in df_train.columns:
#     if np.sum(df_train[val].isnull()) > length_df:
#         list_temp.append(val)   
# df_train = df_train.drop(list_temp, axis=1)

In [25]:
# list(df_train.columns)